"FC layers referenced from https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65"


In [56]:
import torch
import os
import numpy as np
import pickle
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
from sklearn.manifold import TSNE
import plotly.io as pio
from sklearn.utils import class_weight
import tqdm as notebook_tqdm
from tqdm import tqdm

In [57]:
# !pip install ipywidgets

<h3> Declare functions

In [58]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.fc(x)
        return x

class ActivationLayer(nn.Module):
    def __init__(self, activation_fn):
        super(ActivationLayer, self).__init__()
        self.activation_fn = activation_fn

    def forward(self, x):
        x = self.activation_fn(x)
        return x

def tanh(x):
    return torch.tanh(x)

def sigmoid(x):
    return torch.sigmoid(x)

class MyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyNetwork, self).__init__()
        self.fc1 = FCLayer(input_dim, hidden_dim)
        self.activation1 = ActivationLayer(tanh)
        self.fc2 = FCLayer(hidden_dim, output_dim)
        self.activation2 = ActivationLayer(sigmoid)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        x = self.activation2(x)
        return x

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size


In [59]:
# Function to balance class distribution using oversampling
def oversample_data(X_train, Y_train, num_classes):
    # Determine the class with the maximum number of instances
    max_class_count = np.max(np.bincount(Y_train))
    # Generate indices for oversampling each class
    indices_list = [np.where(Y_train == i)[0] for i in range(num_classes)]
    # Oversample minority classes to match the count of the majority class
    for i, indices in enumerate(indices_list):
        if len(indices) < max_class_count:
            oversampled_indices = np.random.choice(indices, size=max_class_count - len(indices), replace=True)
            X_train = np.concatenate((X_train, X_train[oversampled_indices]), axis=0)
            Y_train = np.concatenate((Y_train, Y_train[oversampled_indices]), axis=0)
    return X_train, Y_train


<h4> Extract train labels and label encoder

In [60]:
file_path = "data/dump/labels_train.pkl"
with open(file_path, 'rb') as file:
    y_train = pickle.load(file)
y_train = torch.tensor(y_train)
    
file_path = 'data/dump/label_decoder.pkl'
with open(file_path, 'rb') as file:
    label_decoder = pickle.load(file)

<h4> Extract test labels

In [61]:
file_path = "data/dump/labels_test.pkl"
with open(file_path, 'rb') as file:
    file = open('data/dump/labels_test.pkl', 'rb')
    y_test = pickle.load(file)
y_test = torch.tensor(y_test)

<h4> Getting BERT and GAT outputs for train

In [62]:
file_path = 'embed/u_prime_BERT_train.pkl'
with open(file_path, 'rb') as file:
    u_primes = pickle.load(file)

    concatenated_tensors = []
    for dialogue_tensor in u_primes:
        concatenated_tensors.extend(dialogue_tensor)

    tensorUtterancesTrain = torch.stack(concatenated_tensors)

file_path = "embed/h_prime_BERT-GAT_train.pkl"
with open(file_path, 'rb') as file:
    cherryPickedNodesTrain, _ = pickle.load(file)

file_path = "embed/h_prime_BERT-EGAT_train.pkl"
with open(file_path, 'rb') as file:
    allNodeFeatsTrain, _ = pickle.load(file)

<h4> Getting BERT and GAT outputs for test set

In [63]:
file_path = 'embed/u_prime_BERT_test.pkl'
with open(file_path, 'rb') as file:
    u_primes = pickle.load(file)

    concatenated_tensors = []
    for dialogue_tensor in u_primes:
        concatenated_tensors.extend(dialogue_tensor)

    tensorUtterancesTest = torch.stack(concatenated_tensors)

file_path = "embed/h_prime_BERT-GAT_test.pkl"
with open(file_path, 'rb') as file:
    cherryPickedNodesTest, _ = pickle.load(file)

file_path = "embed/h_prime_BERT-EGAT_test.pkl"
with open(file_path, 'rb') as file:
    allNodeFeatsTest, _ = pickle.load(file)
    
_ = None

<h4> Getting BERT and GAT outputs for the valid set

In [64]:
# TODO do the same code as above once you have u' and h' of valid set

EDA

In [65]:
# # Checking the structure of graph
# for n in range(10):
#     tensor_data_np = tensor_utterances[n].detach().numpy()

#     # Plot the data
#     plt.figure(figsize=(10, 5))
#     plt.plot(range(len(tensor_data_np)), tensor_data_np)
#     plt.title('Line Graph of Tensor Data')
#     plt.xlabel('Index')
#     plt.ylabel('Value')
#     plt.show()


In [66]:
# # Normalize the h' (1st GAT)
# data = cherry_picked_nodes.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [67]:
# # Normalize the h' (2nd GAT)
# data = all_node_feats.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# # Print or analyze the similarity matrix
# # print(similarities)
# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


In [68]:
# # Normalize the u' or updated_representations
# data = tensor_utterances.detach().numpy()
# data_normalized = data / np.linalg.norm(data, axis=1, keepdims=True)

# # Compute pairwise cosine similarities
# similarities = cosine_similarity(data_normalized)

# plt.hist(similarities.flatten(), bins=50, density=True)
# plt.title('Distribution of Cosine Similarities')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.show()


<h3> Feature Selection

In [69]:
# X_train = tensor_utterances
# Y_train = y_train

Part 1

In [70]:
# # Define the number of features (k) to select
# k = 100  # Adjust this value as needed

# # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
# selector = SelectKBest(score_func=f_classif, k=k)

# # Fit SelectKBest on the training data and target variable
# selector.fit(X_train, Y_train)

# # Get the indices of the selected features
# selected_indices = selector.get_support(indices=True)

# # Get the scores of the selected features
# feature_scores = selector.scores_[selected_indices]

# # Display the scores along with their corresponding indices
# # for idx, score in zip(selected_indices, feature_scores):
# #     print(f"Feature index: {idx}, Score: {score}")

# X_train_selected = X_train[:, selected_indices]
# print(X_train_selected.shape)

Pass u' (BERT) and h' (GAT or EGAT) into this method

In [81]:
def get_norm_features(encoded_features):
    scaler = MinMaxScaler()
#       "FeatureSelected+BERT+GAT: ", concatenatedRepresentationTrain2.shape, "\n",
    features_scaled = scaler.fit_transform(encoded_features.clone().detach())
    return torch.tensor(features_scaled)

def get_selected_features(encoded_features, labels, top_n):
    # Apply Min-Max scaling to make the data non-negative
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(encoded_features)

    # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
    selector = SelectKBest(score_func=f_classif, k=100)
    # Assuming feature is your feature matrix (12840 instances x 300 dimensions)
    # and y_train is your target labels

    # Initialize a dictionary to store the indices of top features for each class
    top_features_by_class = {}
    top_scores = {}
    # Calculate the relevance of each feature to each class using chi-squared test
    for label in range(7):  # Assuming you have 7 classes
        # Create a binary mask indicating instances belonging to the current class
        mask = (labels == label)

        # SelectKBest with chi2 as the scoring function
        selector = SelectKBest(score_func=chi2, k=top_n)  # Select top 20 features
        selector.fit(features_scaled, mask)  # Fit SelectKBest to the data
        # Get the indices of the top 20 features
        top_features_indices = np.argsort(selector.scores_)[-top_n:]
        scores = selector.scores_[top_features_indices]
        # Store the indices in the dictionary
        top_features_by_class[label] = top_features_indices
        top_scores[label] = scores

    # Print the top features for each class
    # for label, indices in top_features_by_class.items():
    #     print(f"Label {label_decoder[label]}: idx {', '.join(map(str, indices))}")
    #     print(top_scores[label])

    concatenated_features_set = set()
    for label, indices in top_features_by_class.items():
        concatenated_features_set.update(indices)

    concatenated_features_indices = list(concatenated_features_set)

    concatenated_features_indices = np.array(concatenated_features_indices)

    # Select the desired features
    selected_features = encoded_features[:, concatenated_features_indices]
#     print(selected_features.shape)
    return selected_features, concatenated_features_indices

Selected h'

In [72]:
# X_train = all_node_feats

In [73]:
# # Apply Min-Max scaling to make the data non-negative
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# # Initialize SelectKBest with the desired score function (e.g., f_classif for classification tasks)
# selector = SelectKBest(score_func=f_classif, k=100)
# # Assuming X_train is your feature matrix (12840 instances x 300 dimensions)
# # and y_train is your target labels

# # Initialize a dictionary to store the indices of top features for each class
# top_features_by_class = {}
# top_scores = {}
# # Calculate the relevance of each feature to each class using chi-squared test
# for label in range(7):  # Assuming you have 7 classes
#     # Create a binary mask indicating instances belonging to the current class
#     mask = (Y_train == label)

#     # SelectKBest with chi2 as the scoring function
#     selector = SelectKBest(score_func=chi2, k=20)  # Select top 20 features
#     selector.fit(X_train_scaled, mask)  # Fit SelectKBest to the data
#     # Get the indices of the top 20 features
#     top_features_indices = np.argsort(selector.scores_)[-20:]
#     scores = selector.scores_[top_features_indices]
#     # Store the indices in the dictionary
#     top_features_by_class[label] = top_features_indices
#     top_scores[label] = scores
    
# # Print the top features for each class
# for label, indices in top_features_by_class.items():
#     print(f"Label {label_decoder[label]}: idx {', '.join(map(str, indices))}")
#     print(top_scores[label])

In [74]:
# selected_features1[0]

In [75]:
# selected_features2[0]

In [76]:
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(selected_features.detach().numpy())

# # Plot the PCA result with color-coded labels
# plt.figure(figsize=(8, 6))
# for label in np.unique(Y_train):
#     indices = Y_train == label
#     plt.scatter(pca_result[indices, 0], pca_result[indices, 1], label=f'{label_decoder[label]}', alpha=0.5)
#     plt.title('PCA Visualization of Selected Utterance Embeddings (Train) with Color-Coded Labels')
#     plt.xlabel('Principal Component 1')
#     plt.ylabel('Principal Component 2')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

3d plottly

In [77]:
# X_train = selected_features
# X_train = X_train / np.linalg.norm(X_train, axis=1, keepdims=True)
# # Perform T-SNE dimensionality reduction
# tsne = TSNE(n_components=3, random_state=42)
# X_tsne = tsne.fit_transform(X_train)

# # Create a Plotly scatter plot
# fig = go.Figure(data=[go.Scatter3d(
#     x=X_tsne[:, 0],
#     y=X_tsne[:, 1],
#     z=X_tsne[:, 2],
#     mode='markers',
#     marker=dict(
#         size=3,
#         color=Y_train,  # Assuming Y_train contains labels for coloring
#         colorscale='Viridis',  # You can choose a different colorscale
#         opacity=0.8
#     )
# )])

# # Update layout
# fig.update_layout(title='3D T-SNE Plot', autosize=False,
#                   width=800, height=800)

# # Show the plot
# fig.show()

In [78]:
# Save the plot as an HTML file
# pio.write_html(fig, '3d_tsne_plot.html')

Selected features of train data

In [82]:
# BERT+EGAT
# selectedUPrime, BERT_trainIndices = get_selected_features(tensorUtterancesTrain, y_train, 20)
# selectedHPrime, GAT_trainIndices1 = get_selected_features(allNodeFeatsTrain, y_train, 20)
# concatenatedRepresentationTrain1 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# BERT+GAT
# selectedHPrime, GAT_trainIndices2 = get_selected_features(allNodeFeatsTrain, y_train, 20)
# concatenatedRepresentationTrain2 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# raw-BERT
rawCtxRepresentationTrain = get_norm_features(tensorUtterancesTrain)
# selected-BERT
# ctxRepresentationTrain = selectedUPrime

# raw-BERT+GAT 
concatenatedRepresentationTrain1 = torch.cat((get_norm_features(tensorUtterancesTrain), 
                                              get_norm_features(cherryPickedNodesTrain)), dim=1)
# raw-BERT+EGAT
concatenatedRepresentationTrain2 = torch.cat((get_norm_features(tensorUtterancesTrain), 
                                              get_norm_features(allNodeFeatsTrain)), dim=1)

# print("Sizes of different combination of train data\n",
#       "FeatureSelected+BERT+EGAT: ", concatenatedRepresentationTrain1.shape, "\n",
#       "FeatureSelected+BERT+GAT: ", concatenatedRepresentationTrain2.shape, "\n",
#       "BERT: ", rawCtxRepresentationTrain.shape, "\n",
#       "FeatureSelected+BERT: ", ctxRepresentationTrain.shape)

Selected features of test data

In [83]:
# BERT+EGAT
# selectedUPrime = tensorUtterancesTest[:, BERT_trainIndices]
# selectedHPrime = allNodeFeatsTest[:, GAT_trainIndices1]
# concatenatedRepresentationTest1 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# BERT+GAT
# selectedHPrime = allNodeFeatsTest[:, GAT_trainIndices2]
# concatenatedRepresentationTest2 = torch.cat((selectedUPrime, selectedHPrime), dim=1)
# raw-BERT
rawCtxRepresentationTest = get_norm_features(tensorUtterancesTest)
# selected-BERT
# ctxRepresentationTest = selectedUPrime
# raw-BERT+GAT 
concatenatedRepresentationTest1 = torch.cat((get_norm_features(tensorUtterancesTest), 
                                             get_norm_features(cherryPickedNodesTest)), dim=1)
# raw-BERT+EGAT
concatenatedRepresentationTest2 = torch.cat((get_norm_features(tensorUtterancesTest), 
                                             get_norm_features(allNodeFeatsTest)), dim=1)

# print("Sizes of different combination of train data\n",
#       "FeatureSelected+BERT+EGAT: ", concatenatedRepresentationTest1.shape, "\n",
#       "FeatureSelected+BERT+GAT: ", concatenatedRepresentationTest2.shape, "\n",
#       "BERT: ", rawCtxRepresentationTest.shape, "\n",
#       "FeatureSelected+BERT: ", ctxRepresentationTest.shape)

1. Prep data - normalize and create data loader

In [84]:
def prep_data(features, labels, isTrain):
    num_instances = len(features)
    num_classes = 7

    # Rescale input features
    # selected_features = concatenated_representation / np.linalg.norm(concatenated_representation, axis=1, keepdims=True)

    # Apply data resampling (oversampling) to balance class distribution
    if isTrain:
        X_set, Y_set = oversample_data(features, labels, num_classes)
    else:
        X_set, Y_set = features, labels

    # Calculate class weights for class weighting
#     class_counts = np.bincount(labels)
#     total_instances = np.sum(class_counts)
    # class_weights = torch.tensor([total_instances / (num_classes * count) for count in class_counts], dtype=torch.float32)

    # Convert data to PyTorch tensors
    X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
    Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()
    # print(X_train_tensor.shape, Y_train_tensor.shape)
    # X_train_tensor = torch.tensor(selected_features)
    # Y_train_tensor = torch.tensor(y_train)

    unique_labels, label_counts = np.unique(Y_set, return_counts=True)

    # Print the counts for each unique label
    for label, count in zip(unique_labels, label_counts):
        print(f"Label {label_decoder[label]}: {count} occurrences")

    print(X_tensor.shape, Y_tensor.shape)
    # Create a TensorDataset
    dataset = TensorDataset(X_tensor, Y_tensor)

    return X_tensor, Y_tensor

2. Training

In [85]:
def model_train(X_set, Y_set, input_dim, hidden_dim, num_epochs, num_classes):
    # Initialize the model
    model = MyNetwork(input_dim, hidden_dim, num_classes)
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # Train the model
    print_interval = 1  # Print tqdm every epoch
    for epoch in range(num_epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_instances = 0
        # Use tqdm for progress tracking
        with tqdm(total=len(X_set), desc=f'Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
            for inputs, labels in zip(X_set, Y_set):
                inputs = torch.tensor(inputs.clone().detach(), dtype=torch.float32)
                labels = torch.tensor(labels.clone().detach(), dtype=torch.long)
                # Forward pass
                outputs = model(inputs.unsqueeze(0))  # Add batch dimension
                loss = criterion(outputs, labels.unsqueeze(0))  # Add batch dimension to labels

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_instances += 1
                pbar.update(1)

        # Print average loss and accuracy per epoch
        if (epoch + 1) % print_interval == 0:
            epoch_loss = total_loss / total_instances
            epoch_accuracy = correct_predictions / total_instances
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

    return model


In [ ]:
def classify_emotions(model, X_tensor, Y_tensor, isTrain):
    # Set the model to evaluation mode
    model.eval()

    # Predict on the training data
    with torch.no_grad():
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)

    # Convert predicted tensor to numpy array
    predicted = predicted.numpy()

    # Calculate F1 score per class
    f1_per_class = f1_score(Y_tensor, predicted, average=None)
    f1 = f1_score(Y_tensor, predicted, average='macro')
    if isTrain:
        print(f'Train F1 Score: {f1:.4f}')
    else:
        print(f'Test F1 Score: {f1:.4f}')

    unique_labels, label_counts = np.unique(predicted, return_counts=True)

    # Print F1 score for each class
    for i, f1 in enumerate(f1_per_class):
        print(f'F1 Score for Class {label_decoder[i]}: {f1:.4f}')

    # Print the counts for each unique label
    for label, count in zip(unique_labels, label_counts):
        print(f"Label {label_decoder[label]}: {count} occurrences")

In [ ]:
# TODO create table of loss and accuracy during training
# TODO also compute the time it takes to complete the train

<h4> Train and validate BERT+EGAT given 50 epochs

In [87]:
X_trainTensor, Y_trainTensor = prep_data(concatenatedRepresentationTrain2, y_train, False)
fcClf = model_train(X_set=X_trainTensor, 
                    Y_set=Y_trainTensor,
                    input_dim=X_trainTensor.shape[1], 
                    hidden_dim=256, 
                    num_epochs=30, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


Label anger: 1500 occurrences
Label disgust: 364 occurrences
Label fear: 338 occurrences
Label joy: 2312 occurrences
Label neutral: 5960 occurrences
Label sadness: 876 occurrences
Label surprise: 1490 occurrences
torch.Size([12840, 1536]) torch.Size([12840])


Epoch 1/50:   0%|                                                                            | 0/12840 [00:00<?, ?it/s]C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs.clone().detach(), dtype=torch.float32)
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels.clone().detach(), dtype=torch.long)


Epoch [1/50], Loss: 1.6696, Accuracy: 0.4655


Epoch [2/50], Loss: 1.5932, Accuracy: 0.5313


...

Epoch [46/50], Loss: 1.4890, Accuracy: 0.6296


Epoch [47/50], Loss: 1.4883, Accuracy: 0.6298


Epoch [48/50], Loss: 1.4871, Accuracy: 0.6314


Epoch [49/50], Loss: 1.4868, Accuracy: 0.6298


Epoch [50/50], Loss: 1.4862, Accuracy: 0.6303
Train F1 Score: 0.3182
F1 Score for Class anger: 0.4278
F1 Score for Class disgust: 0.0000
F1 Score for Class fear: 0.0000
F1 Score for Class joy: 0.5429
F1 Score for Class neutral: 0.7535
F1 Score for Class sadness: 0.0000
F1 Score for Class surprise: 0.5034
Label anger: 917 occurrences
Label joy: 2079 occurrences
Label neutral: 9010 occurrences
Label surprise: 834 occurrences


In [93]:
# Predict the test set
X_testTensor, Y_testTensor = prep_data(concatenatedRepresentationTest2, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 1536]) torch.Size([3400])
Test F1 Score: 0.1267
F1 Score for Class anger: 0.0882
F1 Score for Class disgust: 0.0000
F1 Score for Class fear: 0.0000
F1 Score for Class joy: 0.1500
F1 Score for Class neutral: 0.5642
F1 Score for Class sadness: 0.0000
F1 Score for Class surprise: 0.0849
Label anger: 210 occurrences
Label joy: 652 occurrences
Label neutral: 2348 occurrences
Label surprise: 190 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


<h4> Train and validate BERT+GAT given 2 epoch

In [100]:
X_trainTensor, Y_trainTensor = prep_data(concatenatedRepresentationTrain1, y_train, True)
fcClf = model_train(X_set=X_trainTensor, 
                    Y_set=Y_trainTensor,
                    input_dim=X_trainTensor.shape[1], 
                    hidden_dim=256, 
                    num_epochs=3, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


Label anger: 1500 occurrences
Label disgust: 364 occurrences
Label fear: 338 occurrences
Label joy: 2312 occurrences
Label neutral: 5960 occurrences
Label sadness: 876 occurrences
Label surprise: 1490 occurrences
torch.Size([12840, 1536]) torch.Size([12840])


Epoch 1/3:   0%|                                                                             | 0/12840 [00:00<?, ?it/s]C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs.clone().detach(), dtype=torch.float32)
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels.clone().detach(), dtype=torch.long)


KeyboardInterrupt: 

In [96]:
# Predict the test set
X_testTensor, Y_testTensor = prep_data(concatenatedRepresentationTest1, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 1536]) torch.Size([3400])
Test F1 Score: 0.1289
F1 Score for Class anger: 0.1142
F1 Score for Class disgust: 0.0000
F1 Score for Class fear: 0.0000
F1 Score for Class joy: 0.1230
F1 Score for Class neutral: 0.5940
F1 Score for Class sadness: 0.0000
F1 Score for Class surprise: 0.0713
Label anger: 237 occurrences
Label joy: 367 occurrences
Label neutral: 2671 occurrences
Label surprise: 125 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


<h4> Train and validate BERT (x feature selection) given 2 epochs

In [97]:
X_trainTensor, Y_trainTensor = prep_data(rawCtxRepresentationTrain, y_train, False)
fcClf = model_train(X_set=X_trainTensor, 
                    Y_set=Y_trainTensor,
                    input_dim=X_trainTensor.shape[1], 
                    hidden_dim=256, 
                    num_epochs=5, 
                    num_classes=7)
classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


Label anger: 1500 occurrences
Label disgust: 364 occurrences
Label fear: 338 occurrences
Label joy: 2312 occurrences
Label neutral: 5960 occurrences
Label sadness: 876 occurrences
Label surprise: 1490 occurrences
torch.Size([12840, 768]) torch.Size([12840])


Epoch 1/5:   0%|                                                                             | 0/12840 [00:00<?, ?it/s]C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs.clone().detach(), dtype=torch.float32)
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3023422335.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels.clone().detach(), dtype=torch.long)


Epoch [1/5], Loss: 1.6455, Accuracy: 0.4814


Epoch [2/5], Loss: 1.5698, Accuracy: 0.5593


Epoch [3/5], Loss: 1.5531, Accuracy: 0.5776


Epoch [4/5], Loss: 1.5454, Accuracy: 0.5844


Epoch [5/5], Loss: 1.5399, Accuracy: 0.5887
Train F1 Score: 0.1568
F1 Score for Class anger: 0.1164
F1 Score for Class disgust: 0.0000
F1 Score for Class fear: 0.0000
F1 Score for Class joy: 0.2452
F1 Score for Class neutral: 0.6594
F1 Score for Class sadness: 0.0000
F1 Score for Class surprise: 0.0766
Label anger: 184 occurrences
Label joy: 526 occurrences
Label neutral: 12054 occurrences
Label surprise: 76 occurrences


In [98]:
# Predict the test set
X_testTensor, Y_testTensor = prep_data(rawCtxRepresentationTest, y_test, False)
classify_emotions(fcClf, X_testTensor, Y_testTensor, False)

Label anger: 516 occurrences
Label disgust: 99 occurrences
Label fear: 60 occurrences
Label joy: 495 occurrences
Label neutral: 1615 occurrences
Label sadness: 263 occurrences
Label surprise: 352 occurrences
torch.Size([3400, 768]) torch.Size([3400])
Test F1 Score: 0.1132
F1 Score for Class anger: 0.0524
F1 Score for Class disgust: 0.0000
F1 Score for Class fear: 0.0000
F1 Score for Class joy: 0.1091
F1 Score for Class neutral: 0.6159
F1 Score for Class sadness: 0.0000
F1 Score for Class surprise: 0.0153
Label anger: 95 occurrences
Label joy: 275 occurrences
Label neutral: 2990 occurrences
Label surprise: 40 occurrences


C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_set.clone().detach(), dtype=torch.float32).clone().detach()
C:\Users\edayo\AppData\Local\Temp\ipykernel_9580\3259066277.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_tensor = torch.tensor(Y_set.clone().detach(), dtype=torch.long).clone().detach()


<h4> Train and validate BERT (o feature selection) given 2 epochs

In [ ]:
# dataLoader, X_trainTensor, Y_trainTensor = get_data_loader(ctxRepresentationTrain, y_train, True)
# # TODO 3rd argument is tunable
# fcClf = model_train(dataLoader, 
#                     input_dim=X_trainTensor.shape[1], 
#                     output_dim=50, 
#                     num_epochs=2, 
#                     num_classes=7)
# classify_emotions(fcClf, X_trainTensor, Y_trainTensor, True)

In [ ]:
# # Predict the test set
# _, X_testTensor, Y_testTensor = get_data_loader(ctxRepresentationTest, y_test, False)
# classify_emotions(fcClf, X_testTensor, Y_testTensor, False)